# Немного о многоруких бандитах

В ноутбуке про многоруких бандитов мы разбирали классы для реализации отдельных стратегий решения задачки. Реализуйте в той же парадигме стратегии [exp3](https://jamesrledoux.com/algorithms/bandit-algorithms-epsilon-ucb-exp-python/) и [softmax](https://gibberblot.github.io/rl-notes/single-agent/multi-armed-bandits.html) стратегии. Да, вы не ошиблись, они уже сделаны, надо понять, как оно работает и сделать так, чтобы оно работало в использованной нами конфигурации)

В конце концов, вы ж на работе тоже нон-стоп гуглите)


In [1]:
from multi_armed_bandits import BernoulliBandit, Solver
import numpy as np


class Exp3Solver(Solver):
    def __init__(self, bandit, gamma):
        super().__init__(bandit)
        self.gamma = gamma
        self.weights = np.ones(self.bandit.n)

    @property
    def estimated_probas(self):
        weight_sum = np.sum(self.weights)
        return (1 - self.gamma) * (self.weights / weight_sum) + (self.gamma / self.bandit.n)

    def run_one_step(self):
        probas = self.estimated_probas
        chosen_arm = np.random.choice(self.bandit.n, p=probas)

        reward = self.bandit.generate_reward(chosen_arm)
        estimated_reward = reward / probas[chosen_arm]
        self.weights[chosen_arm] *= np.exp(self.gamma * estimated_reward / self.bandit.n)

        return chosen_arm


class SoftmaxSolver(Solver):
    def __init__(self, bandit, tau=1.0):
        super().__init__(bandit)
        self.tau = tau
        self.q_values = np.zeros(self.bandit.n)
        self.counts = np.zeros(self.bandit.n)

    @property
    def estimated_probas(self):
        exp_values = np.exp(self.q_values / self.tau)
        return exp_values / np.sum(exp_values)

    def run_one_step(self):
        probas = self.estimated_probas
        chosen_arm = np.random.choice(self.bandit.n, p=probas)

        reward = self.bandit.generate_reward(chosen_arm)
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]
        self.q_values[chosen_arm] += (reward - self.q_values[chosen_arm]) / n

        return chosen_arm

In [2]:
n_arms = 10
gamma = 0.1
tau = 1.0
num_steps = 1000

bandit = BernoulliBandit(n_arms)

solver = Exp3Solver(bandit, gamma)
solver.run(num_steps)
print("EXP3      Регрет:", round(solver.regret, 3))
print("EXP3      Распределение выбора ручек:", solver.counts)

softmax_solver = SoftmaxSolver(bandit, tau)
softmax_solver.run(num_steps)
print("Softmax   Регрет:", round(softmax_solver.regret, 3))
print("Softmax   Распределение выбора ручек:", softmax_solver.counts)

EXP3      Регрет: 218.215
EXP3      Распределение выбора ручек: [287, 273, 53, 48, 21, 42, 183, 18, 22, 53]
Softmax   Регрет: 357.304
Softmax   Распределение выбора ручек: [338. 298. 166. 188.  98. 178. 342.  96.  82. 214.]


# Про анализ данных

Дан файл [reverb_aggregated.csv](https://github.com/pileyan/DA_samolet_2024/blob/master/lect2.%20Approx.%20quantiles.%20DA%20example.%20LinAlg%20example/reverb_aggregated.csv), используя доступные вам методы анализа данных провести исследование и ответить на вопрос "что влияет на цену инструмента"

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/pileyan/DA_samolet_2024/refs/heads/master/lect2.%20Approx.%20quantiles.%20DA%20example.%20LinAlg%20example/reverb_aggregated.csv")

In [5]:
df.head()

,title,description,text,param_dict,price,listing_id
0,Fender American Standard Stratocaster [XIX19] ...,Welcome to Qsic's Reverb Shop!1. The descripti...,Welcome to Qsic's Reverb Shop!1. The descripti...,{'Condition': 'Very Good (Used)Very Good items...,"$1,131.92",cbbfd652-ca5f-11ed-8644-acde48001122
1,"Fender ""American Vintage II 1951 Telecaster"" 3...",The Fender® American Vintage II series present...,The Fender® American Vintage II series present...,{'Condition': 'Brand New (New)Brand New items ...,"$1,935.55",cd998c16-ca5f-11ed-8644-acde48001122
2,Rebel Custom Guitars Funky Claude 2022 - Hydro...,Stratocaster Style Custom Guitar by Rebel Cust...,Stratocaster Style Custom Guitar by Rebel Cust...,{'Condition': 'Brand New (New)Brand New items ...,"$1,055.53",cfe6936a-ca5f-11ed-8644-acde48001122
3,Fender AMERICAN PROFESSIONAL Stratocaster [MJ8...,Welcome to Qsic's Reverb Shop!1. The descripti...,Welcome to Qsic's Reverb Shop!1. The descripti...,{'Condition': 'Very Good (Used)Very Good items...,"$1,290.62",d1e90396-ca5f-11ed-8644-acde48001122
4,G&L S-500 Premium C.F.S [MJ724] | Reverb,Welcome to Qsic's Reverb Shop!1. The descripti...,Welcome to Qsic's Reverb Shop!1. The descripti...,{'Condition': 'Very Good (Used)Very Good items...,$575.55,d3f30786-ca5f-11ed-8644-acde48001122


In [6]:
df["dict"] = df["param_dict"].apply(eval)

In [7]:
df = pd.concat([df.drop(columns=["param_dict", "dict"]), df["dict"].apply(pd.Series)], axis=1)

In [8]:
df.head(3)

,title,description,text,price,listing_id,Condition,Brand,Model,Categories,Year,...,Neck Construction,Number of Frets,Made In,Finish,Product Family,Artist,Frets,Top Material,Finish Features,Model Sub-Family
0,Fender American Standard Stratocaster [XIX19] ...,Welcome to Qsic's Reverb Shop!1. The descripti...,Welcome to Qsic's Reverb Shop!1. The descripti...,"$1,131.92",cbbfd652-ca5f-11ed-8644-acde48001122,Very Good (Used)Very Good items may show a few...,Fender,American Standard Stratocaster with Rosewood F...,Solid Body,2008,...,Bolt-On,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Fender ""American Vintage II 1951 Telecaster"" 3...",The Fender® American Vintage II series present...,The Fender® American Vintage II series present...,"$1,935.55",cd998c16-ca5f-11ed-8644-acde48001122,Brand New (New)Brand New items are sold by an ...,Fender,"""American Vintage II 1951 Telecaster"" HARDCASE...",NaN,NaN,...,NaN,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Rebel Custom Guitars Funky Claude 2022 - Hydro...,Stratocaster Style Custom Guitar by Rebel Cust...,Stratocaster Style Custom Guitar by Rebel Cust...,"$1,055.53",cfe6936a-ca5f-11ed-8644-acde48001122,Brand New (New)Brand New items are sold by an ...,Rebel Custom Guitars,Funky Claude,Solid Body,2022,...,NaN,NaN,Australia,"Hydro Graphic Film, 2k Clear Coat, Natural Nec...",NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df = df.loc[:, df.isna().sum() < df.shape[0] / 2]
df.head(3)

,title,description,text,price,listing_id,Condition,Brand,Model,Categories,Year,Right / Left Handed,Body Shape,Made In,Finish
0,Fender American Standard Stratocaster [XIX19] ...,Welcome to Qsic's Reverb Shop!1. The descripti...,Welcome to Qsic's Reverb Shop!1. The descripti...,"$1,131.92",cbbfd652-ca5f-11ed-8644-acde48001122,Very Good (Used)Very Good items may show a few...,Fender,American Standard Stratocaster with Rosewood F...,Solid Body,2008,Right Handed,S-Style,NaN,NaN
1,"Fender ""American Vintage II 1951 Telecaster"" 3...",The Fender® American Vintage II series present...,The Fender® American Vintage II series present...,"$1,935.55",cd998c16-ca5f-11ed-8644-acde48001122,Brand New (New)Brand New items are sold by an ...,Fender,"""American Vintage II 1951 Telecaster"" HARDCASE...",NaN,NaN,NaN,NaN,United States,NaN
2,Rebel Custom Guitars Funky Claude 2022 - Hydro...,Stratocaster Style Custom Guitar by Rebel Cust...,Stratocaster Style Custom Guitar by Rebel Cust...,"$1,055.53",cfe6936a-ca5f-11ed-8644-acde48001122,Brand New (New)Brand New items are sold by an ...,Rebel Custom Guitars,Funky Claude,Solid Body,2022,NaN,NaN,Australia,"Hydro Graphic Film, 2k Clear Coat, Natural Nec..."


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31920 entries, 0 to 31919
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                31920 non-null  object
 1   description          31920 non-null  object
 2   text                 31920 non-null  object
 3   price                31920 non-null  object
 4   listing_id           31920 non-null  object
 5   Condition            31920 non-null  object
 6   Brand                31920 non-null  object
 7   Model                31920 non-null  object
 8   Categories           23991 non-null  object
 9   Year                 19030 non-null  object
 10  Right / Left Handed  18929 non-null  object
 11  Body Shape           16140 non-null  object
 12  Made In              19902 non-null  object
 13  Finish               24199 non-null  object
dtypes: object(14)
memory usage: 3.4+ MB


In [11]:
# df["Year"].astype("float64")
df["Categories"].fillna("Unknown", inplace=True)
df["Right / Left Handed"].fillna("Unknown", inplace=True)
df["Body Shape"].fillna("Unknown", inplace=True)
df["Made In"].fillna("Unknown", inplace=True)
df["Finish"].fillna("Unknown", inplace=True)
df["Year"].fillna("Unknown", inplace=True)

df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

<>:9: SyntaxWarning: invalid escape sequence '\$'
<>:9: SyntaxWarning: invalid escape sequence '\$'
C:\Users\chizh\AppData\Local\Temp\ipykernel_23828\4236843794.py:9: SyntaxWarning: invalid escape sequence '\$'
  df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)


In [12]:
df.head(3)

,title,description,text,price,listing_id,Condition,Brand,Model,Categories,Year,Right / Left Handed,Body Shape,Made In,Finish
0,Fender American Standard Stratocaster [XIX19] ...,Welcome to Qsic's Reverb Shop!1. The descripti...,Welcome to Qsic's Reverb Shop!1. The descripti...,1131.92,cbbfd652-ca5f-11ed-8644-acde48001122,Very Good (Used)Very Good items may show a few...,Fender,American Standard Stratocaster with Rosewood F...,Solid Body,2008,Right Handed,S-Style,Unknown,Unknown
1,"Fender ""American Vintage II 1951 Telecaster"" 3...",The Fender® American Vintage II series present...,The Fender® American Vintage II series present...,1935.55,cd998c16-ca5f-11ed-8644-acde48001122,Brand New (New)Brand New items are sold by an ...,Fender,"""American Vintage II 1951 Telecaster"" HARDCASE...",Unknown,Unknown,Unknown,Unknown,United States,Unknown
2,Rebel Custom Guitars Funky Claude 2022 - Hydro...,Stratocaster Style Custom Guitar by Rebel Cust...,Stratocaster Style Custom Guitar by Rebel Cust...,1055.53,cfe6936a-ca5f-11ed-8644-acde48001122,Brand New (New)Brand New items are sold by an ...,Rebel Custom Guitars,Funky Claude,Solid Body,2022,Unknown,Unknown,Australia,"Hydro Graphic Film, 2k Clear Coat, Natural Nec..."


In [13]:
df.loc[4, "description"]

"Welcome to Qsic's Reverb Shop!1. The description texts have been generated from our Japanese site by automatic translation.Instrument or music terms are often not translated correctly, so please let us know if you have any questions.2. For orders from countries or areas where EMS does not support..."

In [14]:
df.loc[4, "text"]

"Welcome to Qsic's Reverb Shop!1. The description texts have been generated from our Japanese site by automatic translation.Instrument or music terms are often not translated correctly, so please let us know if you have any questions.2. For orders from countries or areas where EMS does not support, the difference in shipping costs will be charged again via PayPal after ordering.At first, please feel free to order as it is.Product DetailsSpecification:Body: AlderNeck: MapleFingerboard: rosewood[condition] You can see dents and scratches.There are some missing back panels.E Fret remaining: About 70 %E Truss rod: I can afford itE Electrical system: No problemE Weight: 3.9kgE Neck: No problem[Adjustment content]E Overall cleaningE Neck adjustmentE String height adjustmentE Octave adjustmentE Electrical cleaningSerial number:#7100007Our store management number: MJ724AccessoriesSoft case, arm-----------------------------------Please note that all items are for sale in our Store in Kobe.There

In [15]:
df.drop(columns=["description"], inplace=True)

In [16]:
df.loc[4, "title"]

'G&L S-500 Premium C.F.S [MJ724] | Reverb'

In [17]:
df["listing_id"].value_counts()

listing_id
ebd1d780-d516-11ed-8644-acde48001122    3
a421456c-d514-11ed-8644-acde48001122    3
8f92a7c6-d514-11ed-8644-acde48001122    3
9233297e-d514-11ed-8644-acde48001122    3
94894280-d514-11ed-8644-acde48001122    3
                                       ..
4dcfd8ca-ca26-11ed-8644-acde48001122    1
4bf0ad72-ca26-11ed-8644-acde48001122    1
4a17ff14-ca26-11ed-8644-acde48001122    1
47f8b6ce-ca26-11ed-8644-acde48001122    1
9ccd4aa8-d5e2-11ed-8644-acde48001122    1
Name: count, Length: 23340, dtype: int64

In [18]:
print("Shape with duplicates:    ", df.shape)
df.drop_duplicates(inplace=True)
print("Shape without duplicates: ", df.shape)

Shape with duplicates:     (31920, 13)
Shape without duplicates:  (23340, 13)


In [19]:
df.drop(columns=['listing_id'], inplace=True)

In [20]:
df["Year"].value_counts()

Year
Unknown              9275
2022                 2143
2023                 1107
2021                  746
2020                  563
                     ... 
2012-preasnt Day        1
2015- Present Day       1
1983 - 1987             1
?                       1
1998 - 2013             1
Name: count, Length: 891, dtype: int64

In [21]:
def clean_year(year_str):
    if len(year_str) >= 4:
        if year_str[:4].isdigit():
            return year_str[:4]
    
    return "0"

In [22]:
df["Year"] = df["Year"].apply(clean_year).astype(int)
df = df[(df["Year"] > 1900) | (df["Year"] == 0)] # delete 1122 year
mean_year = round(df.loc[df["Year"] != 0, "Year"].mean())
df["Year"] = df["Year"].apply(lambda y: mean_year if y == 0 else y)
df.head()

,title,text,price,Condition,Brand,Model,Categories,Year,Right / Left Handed,Body Shape,Made In,Finish
0,Fender American Standard Stratocaster [XIX19] ...,Welcome to Qsic's Reverb Shop!1. The descripti...,1131.92,Very Good (Used)Very Good items may show a few...,Fender,American Standard Stratocaster with Rosewood F...,Solid Body,2008,Right Handed,S-Style,Unknown,Unknown
1,"Fender ""American Vintage II 1951 Telecaster"" 3...",The Fender® American Vintage II series present...,1935.55,Brand New (New)Brand New items are sold by an ...,Fender,"""American Vintage II 1951 Telecaster"" HARDCASE...",Unknown,2005,Unknown,Unknown,United States,Unknown
2,Rebel Custom Guitars Funky Claude 2022 - Hydro...,Stratocaster Style Custom Guitar by Rebel Cust...,1055.53,Brand New (New)Brand New items are sold by an ...,Rebel Custom Guitars,Funky Claude,Solid Body,2022,Unknown,Unknown,Australia,"Hydro Graphic Film, 2k Clear Coat, Natural Nec..."
3,Fender AMERICAN PROFESSIONAL Stratocaster [MJ8...,Welcome to Qsic's Reverb Shop!1. The descripti...,1290.62,Very Good (Used)Very Good items may show a few...,Fender,American Professional Stratocaster with Rosewo...,Solid Body,2017,Right Handed,S-Style,Unknown,Unknown
4,G&L S-500 Premium C.F.S [MJ724] | Reverb,Welcome to Qsic's Reverb Shop!1. The descripti...,575.55,Very Good (Used)Very Good items may show a few...,G&L,S-500 Premium C.F.S [MJ724],Unknown,2005,Unknown,Unknown,Unknown,Unknown


In [23]:
df["Condition"].value_counts()

Condition
Brand New (New)Brand New items are sold by an authorized dealer or original builder and include all original packaging.learn more                                8876
Very Good (Used)Very Good items may show a few slight marks or scratches but are fully functional and in overall great shape.learn more                          4925
Excellent (Used)Excellent items are almost entirely free from blemishes and other visual defects and have been played or used with the utmost care.learn more    4850
Mint (Used)Mint items are in essentially new original condition but have been opened or played.learn more                                                        2308
Good (Used)Good condition items function properly but may exhibit some wear and tear.learn more                                                                  2008
Fair (Used)Fair condition gear should function but will show noticeable cosmetic damage or other issues.learn more                                              

In [24]:
df["Condition"] = df["Condition"].apply(lambda s: str(s.split(" (")[0]))

In [25]:
for col in df.select_dtypes("object").columns:
    print(col + ":", df[col].nunique())

title: 17809
text: 17330
Condition: 9
Brand: 1442
Model: 11755
Categories: 52
Right / Left Handed: 3
Body Shape: 8
Made In: 55
Finish: 4127


Можем применить OHE к категориальным столбцам Condition, Categories, Right / Left Handed, Body Shape и Made In, потому что у них немного уникальных значений. title и text можем закодировать с помощью TF-IDF. Остальные категориальные признаки закодируем с помощью TargetEncoder.

In [26]:
from sklearn.preprocessing import TargetEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
ohe_columns = ['Condition', 'Categories', 'Right / Left Handed', 'Body Shape', 'Made In']
te_columns = ['Brand', 'Model', 'Finish']

df = pd.get_dummies(df, columns=ohe_columns, drop_first=True, dtype=int)

df[te_columns] = TargetEncoder().fit_transform(df[te_columns], df['price'])

tfidf_vectorizer_title = TfidfVectorizer(max_features=1000, stop_words="english")
tfidf_vectorizer_text = TfidfVectorizer(max_features=1000, stop_words="english")

tfidf_title = tfidf_vectorizer_title.fit_transform(df['title']).toarray()
tfidf_text = tfidf_vectorizer_text.fit_transform(df['text']).toarray()

tfidf_title_df = pd.DataFrame(tfidf_title, columns=[f'Title_TFIDF_{i}' for i in range(tfidf_title.shape[1])])
tfidf_text_df = pd.DataFrame(tfidf_text, columns=[f'Text_TFIDF_{i}' for i in range(tfidf_text.shape[1])])

df = pd.concat([df.reset_index(drop=True), tfidf_title_df, tfidf_text_df], axis=1).drop(columns=["title", "text"])

In [28]:
df.head()

,price,Brand,Model,Year,Finish,Condition_Brand New,Condition_Excellent,Condition_Fair,Condition_Good,Condition_Mint,...,Text_TFIDF_990,Text_TFIDF_991,Text_TFIDF_992,Text_TFIDF_993,Text_TFIDF_994,Text_TFIDF_995,Text_TFIDF_996,Text_TFIDF_997,Text_TFIDF_998,Text_TFIDF_999
0,1131.92,3645.778341,1627.120973,2008,2261.339650,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,1935.55,3645.778341,1932.380000,2005,2261.339650,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.116965,0.0,0.0,0.0
2,1055.53,2003.936139,1059.320000,2022,1059.320000,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,1290.62,3635.551204,1379.063749,2017,2193.171446,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,575.55,1433.515476,574.180000,2005,2295.772536,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


Посмотрим на корреляцию между признаками и таргетом:

In [29]:
corr = df.corr()

In [30]:
abs(corr['price']).sort_values(ascending=False)[1:21]

Model                    0.721469
Brand                    0.298136
Made In_United States    0.244997
Finish                   0.236444
Text_TFIDF_618           0.176154
Title_TFIDF_276          0.174128
Title_TFIDF_369          0.164701
Year                     0.156172
Title_TFIDF_424          0.154920
Text_TFIDF_364           0.143396
Title_TFIDF_28           0.139032
Text_TFIDF_230           0.127268
Text_TFIDF_14            0.117463
Text_TFIDF_509           0.115836
Title_TFIDF_823          0.112500
Made In_Unknown          0.109992
Text_TFIDF_307           0.105793
Title_TFIDF_307          0.104816
Text_TFIDF_727           0.101591
Title_TFIDF_23           0.098890
Name: price, dtype: float64

Как мы видим, цена сильнее всего скоррелирована с моделью товара. Также она коррелирует с брендом, с фактом производства в США, с Finish (что бы это не значило), с годом производства и отдельными словами из названия и описания товара.

Попробуем построить линейную модель и оценить вес каждого признака.

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [32]:
X = df.drop(columns=["price"])
y = df["price"]

X_scaled = StandardScaler().fit_transform(X)
lr = LinearRegression().fit(X_scaled, y)

In [33]:
df_lr = pd.DataFrame(lr.coef_, index=X.columns, columns=["coefs"])

In [34]:
top_lr = abs(df_lr).sort_values(by="coefs", ascending=False).head(20)
top_lr

,coefs
Model,3809.601529
Text_TFIDF_26,801.341123
Year,596.902068
Brand,527.788225
Text_TFIDF_27,462.005705
Text_TFIDF_710,434.484271
Categories_Solid Body,418.404031
Title_TFIDF_23,416.644608
Text_TFIDF_68,397.293174
Categories_Unknown,374.045130


Здесь признак Model тоже является наиболее значимым. Значимость года производства здесь выше, а вот значимость бренда - ниже. Факт производства в США здесь не так важен. Отдельные  слова из описания и названия также сильно влияют на прогноз.

In [35]:
text_indices = []
title_indices = []

for feature in top_lr.index:
    arr = feature.split("_")
    if arr[0] == "Title":
        title_indices.append(int(arr[2]))
    if arr[0] == "Text":
        text_indices.append(int(arr[2]))

Посмотрим на самые значимые слова в названии товара:

In [36]:
for idx in title_indices:
    print(tfidf_vectorizer_title.get_feature_names_out()[idx], end=", ")

1954, 1959, harley, benton, 1956, 

Посмотрим на самые значимые слова в описании товара:

In [37]:
for idx in text_indices:
    print(tfidf_vectorizer_text.get_feature_names_out()[idx], end=", ")

30am, 30pm, professional, adaptor, ishibashis, government, returns, 